In [0]:
### Function that query the image from google street view API, save the image in drive and return the queried image
import cv2
import requests
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

def GetImage_fromAPI(address):
  key = #<Your KEY!!>
  base_url = "https://maps.googleapis.com/maps/api/streetview?size=128x128&location="
  parameter = "&fov=105&pitch=25&source=outdoor&key="
  
  query_url = base_url+address+parameter+key
  
  r = requests.get(query_url)
  i=Image.open(BytesIO(r.content))
  #i = i.convert("RGB")
  i.save("/content/drive/My Drive/CCAI5/"+address.replace(" ","")+".jpg")
  return(i)


#test = GetImage_fromAPI("433 rue Forest Mont-Saint-Hilaire QC J3H 4V2")
#plt.imshow(np.array(test))

def GetImage_fromDrive(address):
  
  filename = "/content/drive/My Drive/CCAI5/"+address.replace(" ","")+".jpg"
  
  im = cv2.imread(filename)
  return(im)

#test = GetImage_fromDrive("49.2886395,-123.13281465")
#print(type(test))
#plt.imshow(test)
#test.max()
  
  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
## Testing ez filtering tool for empty image
np_img = np.array(test)
np_sum = np_img.sum(axis=-1)
print(np_sum[:10])
print(np.all(np_sum[:10]==678))
print("Fuck yeah")

[[678 678 678 ... 678 678 678]
 [678 678 678 ... 678 678 678]
 [678 678 678 ... 678 678 678]
 ...
 [678 678 678 ... 678 678 678]
 [678 678 678 ... 678 678 678]
 [678 678 678 ... 678 678 678]]
True


In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [0]:
#Downloading pre-trained model

import tensorflow as tf
IMG_SHAPE = (128, 128, 3)

# Create the base model from the pre-trained model Resnet-50 or VGG19
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
9412608/9406464 [==============================] - 1s 0us/step


In [0]:
### Generator to process data from query
import logging

def data_generator(dataset, batch_size=50,FromAPI=True):
    """A generator that returns images and corresponding target class ids,
    """
    b = 0  # batch item index
    image_index = -1
    error_count = 0
    dataset_size = len(dataset)

    # Keras requires a generator to run indefinitely.
    while True:
        try:
            # Increment index to pick next image. Shuffle if at the start of an epoch.
            image_index = (image_index + 1) % dataset_size

            # Targets
            numb_floors = dataset.LEVELS[image_index]

            # image input
            coord = str(dataset.LON[image_index])+"," + str(dataset.LAT[image_index])
            if FromAPI:
              img = np.array(GetImage_fromAPI(coord))
            else:
              img = GetImage_fromDrive(coord)
              #if type(img)!='numpy.ndarray':
                #continue
            
            #Skip images that have no instances. Google query will return it as metadate (no img found)
            if img is None:  #if unfindable: download
              img = np.array(GetImage_fromAPI(coord))
              #continue
              
            np_sum = img.sum(axis=-1)
            if np.all(np_sum[:10]==678):
              continue

            # Init batch arrays
            if b == 0:
                
                batch_images = np.zeros(
                    (batch_size,) + img.shape, dtype=np.float32)
                batch_target_class = np.zeros(
                    batch_size, dtype=np.int32)

            # Add to batch
            batch_images[b] = img/255
            batch_target_class[b] = numb_floors
            b += 1

            # Batch full?
            if b >= batch_size:
                inputs = [batch_images]
                outputs = [batch_target_class]

                yield inputs, outputs

                # start a new batch
                b = 0
        except (GeneratorExit, KeyboardInterrupt):
            raise
        except:
            # Log it and skip the image
            logging.exception("Error processing image {}".format(
                dataset.image_info[image_id]))
            error_count += 1
            if error_count > 5:
                raise

In [0]:
## Preparing pre-trained model for fine-tuning
import keras
base_model.trainable = False
#base_model.summary()

model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(11, activation='softmax')
])

model.summary()


Using TensorFlow backend.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_128 (Model) (None, 4, 4, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 11)                14091     
Total params: 2,272,075
Trainable params: 14,091
Non-trainable params: 2,257,984
_________________________________________________________________


In [0]:
#Reload model
import tensorflow as tf
import keras
model = tf.keras.models.load_model('/content/drive/My Drive/MobileNet_v1.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4,clipnorm=1.),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
import pandas as pd
dataset = pd.read_csv("output-final.csv",sep=";",usecols=[0, 1, 2])
display(dataset.head())
dataset.LEVELS = dataset.LEVELS.astype(int)
#shuffle
dataset = dataset.sample(frac=1).reset_index(drop=True)
print(str(dataset.LON[0])+"," + str(dataset.LAT[0]))
len(dataset)

,LAT,LON,LEVELS
0,-80.967173,35.305898,2
1,-80.938653,35.309034,1
2,-80.938848,35.309204,1
3,-80.938531,35.308724,1
4,-80.936607,35.303380,1


39.9560513,-75.16571175


22411

In [0]:
epochs = 5
batch_size = 50
steps_per_epoch = len(dataset)/(10*batch_size)
validation_steps = 1
train_generator = data_generator(dataset[:8000], batch_size=batch_size, FromAPI=False)
valid_generator = data_generator(dataset[8000:].reset_index(drop=True), batch_size=15, FromAPI=False)

history_fine = model.fit_generator(train_generator,
                                   steps_per_epoch = steps_per_epoch,
                                   epochs=epochs,
                                   workers=1,
                                   validation_data=valid_generator,
                                   validation_steps=validation_steps)

Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
17/16 [===============================] - 752s 44s/step - loss: 2.3121 - acc: 0.1800 - val_loss: 2.3861 - val_acc: 0.2667
Epoch 2/5
17/16 [===============================] - 836s 49s/step - loss: 1.9813 - acc: 0.3165 - val_loss: 1.8667 - val_acc: 0.3333
Epoch 3/5
11/16 [===================>..........] - ETA: 6:13 - loss: 1.8548 - acc: 0.3473

KeyboardInterrupt: ignored

In [0]:
#model.save('/content/drive/My Drive/MobileNet_v2.h5')

In [0]:
model.layers[0].trainable = True
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(model.layers[0].layers))

# Fine tune from this layer onwards
fine_tune_at = 0

# Freeze all the layers before the `fine_tune_at` layer
for layer in model.layers[0].layers[:fine_tune_at]:
  layer.trainable =  False
  
model.compile(optimizer=tf.keras.optimizers.Adam(lr=5e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#Retrain with fine-tuning
epochs = 10
batch_size = 50
steps_per_epoch = len(dataset)/(10*batch_size)
validation_steps = 1
train_generator = data_generator(dataset[:22000], batch_size=batch_size, FromAPI=False)
valid_generator = data_generator(dataset[22000:].reset_index(drop=True), batch_size=15, FromAPI=False)

history_fine = model.fit_generator(train_generator,
                                   steps_per_epoch = steps_per_epoch,
                                   epochs=epochs,
                                   workers=1,
                                   validation_data=valid_generator,
                                   validation_steps=validation_steps)

Number of layers in the base model:  87
Epoch 1/10
45/44 [==============================] - 66s 1s/step - loss: 1.6521 - acc: 0.4258 - val_loss: 2.3754 - val_acc: 0.1333
Epoch 2/10
45/44 [==============================] - 728s 16s/step - loss: 1.3973 - acc: 0.4982 - val_loss: 2.4548 - val_acc: 0.2000
Epoch 3/10
45/44 [==============================] - 709s 16s/step - loss: 1.3734 - acc: 0.4929 - val_loss: 1.3623 - val_acc: 0.5333
Epoch 4/10
45/44 [==============================] - 710s 16s/step - loss: 1.2569 - acc: 0.5222 - val_loss: 2.2065 - val_acc: 0.3333
Epoch 5/10
45/44 [==============================] - 706s 16s/step - loss: 1.2059 - acc: 0.5413 - val_loss: 0.7690 - val_acc: 0.6000
Epoch 6/10
45/44 [==============================] - 713s 16s/step - loss: 1.1353 - acc: 0.5644 - val_loss: 1.0804 - val_acc: 0.4667
Epoch 7/10
45/44 [==============================] - 725s 16s/step - loss: 1.1659 - acc: 0.5476 - val_loss: 1.1238 - val_acc: 0.6000
Epoch 8/10
35/44 [====================

In [0]:
#Retrain with fine-tuning
epochs = 10
batch_size = 50
steps_per_epoch = len(dataset)/(10*batch_size)
validation_steps = 1
train_generator = data_generator(dataset[:8000], batch_size=batch_size, FromAPI=False)
valid_generator = data_generator(dataset[8000:].reset_index(drop=True), batch_size=15, FromAPI=False)

history_fine = model.fit_generator(train_generator,
                                   steps_per_epoch = steps_per_epoch,
                                   epochs=epochs,
                                   workers=1,
                                   validation_data=valid_generator,
                                   validation_steps=validation_steps)

Epoch 1/10
17/16 [===============================] - 13s 752ms/step - loss: 0.1674 - acc: 0.9882 - val_loss: 0.7838 - val_acc: 0.6667
Epoch 2/10
17/16 [===============================] - 13s 744ms/step - loss: 0.3231 - acc: 0.9282 - val_loss: 1.0620 - val_acc: 0.8000
Epoch 3/10
17/16 [===============================] - 13s 759ms/step - loss: 0.3358 - acc: 0.9424 - val_loss: 1.6683 - val_acc: 0.5333
Epoch 4/10
17/16 [===============================] - 13s 738ms/step - loss: 0.3568 - acc: 0.9176 - val_loss: 1.6185 - val_acc: 0.4000
Epoch 5/10
17/16 [===============================] - 13s 748ms/step - loss: 0.2918 - acc: 0.9565 - val_loss: 0.6682 - val_acc: 0.6000
Epoch 6/10
17/16 [===============================] - 13s 739ms/step - loss: 0.3094 - acc: 0.9529 - val_loss: 1.3210 - val_acc: 0.4000
Epoch 7/10
17/16 [===============================] - 13s 757ms/step - loss: 0.2680 - acc: 0.9718 - val_loss: 1.0494 - val_acc: 0.8000
Epoch 8/10
17/16 [===============================] - 13s 756ms

In [0]:
#model.save('/content/drive/My Drive/MobileNet2_v1.h5')

False